# NLPTK: тулкит для Natural Language Processing

## Работа с классом Text: основные методы и свойства

In [1]:
# минимально необходимые внешние модули
#!pip install chardet
#!pip install dawg
#!pip install gensim
#!pip install pattern
#!pip install razdel
#!pip install pymorphy2
#!pip install cyhuspell
#!pip install pyenchant

import sys, os
# сменим рабочую директорию на ту, где расположен каталог с библиотекой nlptk
os.chdir(r'F:\INSTALL\Python3\PROJECTS\REPOS\nlp_toolkit')

In [6]:
#from nlptk.mining.utils import datapath

#import nltk
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
# добавим этот путь в sys.path, чтобы интерпретатор увидел нашу библиотеку
sys.path.append(r'F:\INSTALL\Python3\PROJECTS\REPOS\nlp_toolkit')


In [3]:
import sys
from pprint import pprint
import nlptk
from nlptk.mining.text import *



unable to import 'smart_open.gcs', disabling that module


In [5]:
# создаем экземпляр класса Prep c настройками по умолчанию
prep = Prep()
# это то, что будет использоваться при обработке текста
prep.__dict__

{'sentencizer': <function nlptk.misc.mixins.SentencizerMixin.sentencize_nltk(text, *args, lang='english', **kwargs)>,
 'tokenizer': functools.partial(<function TokenizerMixin.toktok_tokenize at 0x0000000012FBA798>, strip='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'),
 'tagger': <function nlptk.misc.mixins.TaggerMixin.tagger_nltk(tokens, *args, lang='eng', **kwargs)>,
 'lemmatizer': functools.partial(<function LemmatizerMixin.lemmatize_nltk at 0x0000000012FBA318>, pos=True, normalize_uppercase=<method 'capitalize' of 'str' objects>),
 'speller': <nlptk.spelling.spellers.Speller at 0x15c8c688>}

In [6]:
# набор правил для очистки текста; применяются на этапе работы с необработанным текстом;
rules_clean = OrderedDict(
    contractions=True,        # замена английских сокращений общего характера (служебные слова) на их полную версию: didn't => did not
    possessive_endings=True,  # удаление окончаний притяжательного падежа в анг.
    #roman_numerals=(True,)    # удаление римских чисел
    # здесь еще много всяких правил используемых по умолчанию
)
# набор правил для фильтра токенов; применяются на этапе запроса слов\токенов\лемм из тэгированного текста
# могут быть включены на уровне составления вокабуляра класса Corpus 
# (по умолчанию фильтрация выключена, поэтому вокабуляр максимально полный)
rules_filter = OrderedDict(
        by_tagpos=(True,{},{"FW","POS"}), # фильтр по тегам частей речи - игнорировать иностранные слова и possessive ending parent's
        punctuation=True, # по умолчанию
        short=(True,3),   # по умолчанию
        stopwords=True,   # по умолчанию
        ifnotin_lexicon=True
        #trailing_chars=True
)

# классы которым передаются правила, сами же экземпляры классов передаются в конструктор класса Text
clean = TextCleaner(rules_clean)
filters = TokenFilter(rules_filter)   



In [8]:
clean.rules

OrderedDict([('hyphenation', (True,)),
             ('accent', (True,)),
             ('contractions', (True,)),
             ('possessive_endings', (True,)),
             ('tags', (True,)),
             ('urls', (True,)),
             ('numeric', (True,)),
             ('roman_numerals', (False,)),
             ('nonletter_sequences', (True,)),
             ('quotes', (True,)),
             ('multiple_whitespaces', (True,))])

In [7]:
filters.rules

In [7]:
# класс Text может принять в качестве ввода непосредственно текст (нужно указать это в параметре input='text', 
# по дефолту input="filename")
intake='''THE PICTURE OF DORIAN GRAY

BY

OSCAR WILDE. The it's artist is the creator of beautiful things. PREFACE'''
text1 = Text(intake, prep, clean, filters, input='text', inplace=True)
print(text1)

THE⁄The⁄DT PICTURE⁄Picture⁄NN OF⁄Of⁄IN DORIAN⁄Dorian⁄JJ GRAY⁄Gray⁄NNP BY⁄By⁄IN OSCAR⁄Oscar⁄NNP WILDE⁄Wilde⁄NNP
The⁄The⁄DT it⁄it⁄PRP is⁄be⁄VBZ artist⁄artist⁄JJ is⁄be⁄VBZ the⁄the⁄DT creator⁄creator⁄NN of⁄of⁄IN beautiful⁄beautiful⁄JJ things⁄thing⁄NNS
PREFACE⁄Preface⁄NN


In [8]:
text1.tags()

[('THE', 'DT', 'The'),
 ('PICTURE', 'NN', 'Picture'),
 ('OF', 'IN', 'Of'),
 ('DORIAN', 'JJ', 'Dorian'),
 ('GRAY', 'NNP', 'Gray'),
 ('BY', 'IN', 'By'),
 ('OSCAR', 'NNP', 'Oscar'),
 ('WILDE', 'NNP', 'Wilde'),
 ('The', 'DT', 'The'),
 ('it', 'PRP', 'it'),
 ('is', 'VBZ', 'be'),
 ('artist', 'JJ', 'artist'),
 ('is', 'VBZ', 'be'),
 ('the', 'DT', 'the'),
 ('creator', 'NN', 'creator'),
 ('of', 'IN', 'of'),
 ('beautiful', 'JJ', 'beautiful'),
 ('things', 'NNS', 'thing'),
 ('PREFACE', 'NN', 'Preface')]

In [14]:
text1.postags(pos='JJ')

FreqDist({'dorian': 1, 'artist': 1, 'beautiful': 1})

In [15]:
text1.postags(pos='VERB')

FreqDist({'be': 2})

In [8]:
text1.sents()

[TaggedSentence(
 	'THE⁄The⁄DT PICTURE⁄Picture⁄NN OF⁄Of⁄IN DORIAN⁄Dorian⁄JJ GRAY⁄Gray⁄NNP BY⁄By⁄IN OSCAR⁄Oscar⁄NNP WILDE⁄Wilde⁄NNP',
 	 n=0
 ),
 TaggedSentence(
 	'The⁄The⁄DT it⁄it⁄PRP is⁄be⁄VBZ artist⁄artist⁄JJ is⁄be⁄VBZ the⁄the⁄DT creator⁄creator⁄NN of⁄of⁄IN beautiful⁄beautiful⁄JJ things⁄thing⁄NNS',
 	 n=1
 ),
 TaggedSentence(
 	'PREFACE⁄Preface⁄NN',
 	 n=2
 )]

In [9]:
text1.sents(1).tokens()[3]

Token(word='artist', idx=3, pos='JJ', lemma='artist', nsent=1)

In [13]:
#help(text1.speller)

In [12]:
text1.speller.spell('hello')


True

In [10]:
text1.speller.suggest('helloo')

('hello', 'hellos', 'hello o', 'hellhole')

In [10]:
text1.speller.replace('hellooя')

'hellooя'

In [17]:
text1.speller().suggest(text1.sents(1).tokens()[3].lemma)

('artist', 'artiste', 'artiest', 'artists')

In [9]:
text1.summarize(3)

[(0.5, 'THE PICTURE OF DORIAN GRAY BY OSCAR WILDE'),
 (0.5, 'The artist is the creator of beautiful things')]

In [7]:
text1.sents(1).tokens()

(Token(word='The', idx=0, pos='DT', lemma='The', nsent=1),
 Token(word='it', idx=1, pos='PRP', lemma='it', nsent=1),
 Token(word='is', idx=2, pos='VBZ', lemma='be', nsent=1),
 Token(word='artist', idx=3, pos='JJ', lemma='artist', nsent=1),
 Token(word='is', idx=4, pos='VBZ', lemma='be', nsent=1),
 Token(word='the', idx=5, pos='DT', lemma='the', nsent=1),
 Token(word='creator', idx=6, pos='NN', lemma='creator', nsent=1),
 Token(word='of', idx=7, pos='IN', lemma='of', nsent=1),
 Token(word='beautiful', idx=8, pos='JJ', lemma='beautiful', nsent=1),
 Token(word='things', idx=9, pos='NNS', lemma='thing', nsent=1))

In [8]:
text1.sents(1).tokens(pos="NN")

(Token(word='creator', idx=6, pos='NN', lemma='creator', nsent=1),
 Token(word='things', idx=9, pos='NNS', lemma='thing', nsent=1))

In [21]:
text1.sents(1).words()

('the',
 'it',
 'is',
 'artist',
 'is',
 'the',
 'creator',
 'of',
 'beautiful',
 'things')

In [9]:
text1.sents(1).words(uniq=True)

('beautiful', 'artist', 'is', 'things', 'creator', 'of', 'the', 'it')

In [10]:
text1.sents(1).text

'The⁄The⁄DT it⁄it⁄PRP is⁄be⁄VBZ artist⁄artist⁄JJ is⁄be⁄VBZ the⁄the⁄DT creator⁄creator⁄NN of⁄of⁄IN beautiful⁄beautiful⁄JJ things⁄thing⁄NNS'

In [11]:
text1.sents(1).raw

'The it is artist is the creator of beautiful things'

In [11]:
text1.words(lower=False, filtrate=False)

['THE',
 'PICTURE',
 'OF',
 'DORIAN',
 'GRAY',
 'BY',
 'OSCAR',
 'WILDE',
 'The',
 'it',
 'is',
 'artist',
 'is',
 'the',
 'creator',
 'of',
 'beautiful',
 'things',
 'PREFACE']

In [12]:
text1.words(lower=True, pos="NNP")

['gray', 'oscar', 'wilde']

In [13]:
text1.words(lower=True, uniq=True)

['artist',
 'wilde',
 'things',
 'preface',
 'oscar',
 'beautiful',
 'dorian',
 'of',
 'by',
 'picture',
 'the',
 'creator',
 'it',
 'gray',
 'is']

In [12]:
text1.sents(0).tags

[('THE', 'DT', 'The'),
 ('PICTURE', 'NN', 'Picture'),
 ('OF', 'IN', 'Of'),
 ('DORIAN', 'JJ', 'Dorian'),
 ('GRAY', 'NNP', 'Gray'),
 ('BY', 'IN', 'By'),
 ('OSCAR', 'NNP', 'Oscar'),
 ('WILDE', 'NNP', 'Wilde')]

In [11]:
text1.count(words=True)

19

In [12]:
text1.count(words=True,token='the')

3

In [13]:
text1.count(words=False, uniq=True)

15

In [10]:
text1.lemmas(uniq=True)

['dorian',
 'the',
 'it',
 'thing',
 'preface',
 'picture',
 'gray',
 'wilde',
 'by',
 'beautiful',
 'artist',
 'creator',
 'be',
 'oscar',
 'of']

In [11]:
text1.lemmas(lower=False, filtrate=False)

['The',
 'Picture',
 'Of',
 'Dorian',
 'Gray',
 'By',
 'Oscar',
 'Wilde',
 'The',
 'it',
 'be',
 'artist',
 'be',
 'the',
 'creator',
 'of',
 'beautiful',
 'thing',
 'Preface']

In [16]:
text1.count(words=False,uniq=True)

15

In [17]:
text1.count(words=True)

19

In [11]:
text1.keywords().topn(7)

['beautiful things',
 'dorian gray',
 'oscar wilde',
 'creator',
 'preface',
 'picture',
 'artist']

In [16]:
text1.trie.items(sort=True)


[('the', (0, 0)),
 ('picture', (0, 1)),
 ('of', (0, 2)),
 ('dorian', (0, 3)),
 ('gray', (0, 4)),
 ('by', (0, 5)),
 ('oscar', (0, 6)),
 ('wilde', (0, 7)),
 ('the', (1, 0)),
 ('it', (1, 1)),
 ('is', (1, 2)),
 ('artist', (1, 3)),
 ('is', (1, 4)),
 ('the', (1, 5)),
 ('creator', (1, 6)),
 ('of', (1, 7)),
 ('beautiful', (1, 8)),
 ('things', (1, 9)),
 ('preface', (2, 0))]

In [23]:
text1.trie.keys()

['artist',
 'beautiful',
 'by',
 'creator',
 'dorian',
 'gray',
 'is',
 'is',
 'it',
 'of',
 'of',
 'oscar',
 'picture',
 'preface',
 'the',
 'the',
 'the',
 'things',
 'wilde']

In [38]:
from collections import defaultdict
sents = defaultdict(list)
for w,(nsent,idx) in trie:
    sents[nsent].append(w)

print(sents)    

defaultdict(<class 'list'>, {0: ['the', 'picture', 'of', 'dorian', 'gray', 'by', 'oscar', 'wilde'], 1: ['the', 'artist', 'is', 'the', 'creator', 'of', 'beautiful', 'things'], 2: ['preface']})


In [18]:
intake = 'The Picture of Dorian Gray By Oscar Wilde'
text2 = Text(intake, prep, clean, filters, input='text', inplace=True)
print(text2)

The⁄The⁄DT Picture⁄Picture⁄NN of⁄of⁄IN Dorian⁄Dorian⁄JJ Gray⁄Gray⁄NNP By⁄By⁄IN Oscar⁄Oscar⁄NNP Wilde⁄Wilde⁄NNP


In [19]:
text2.sents(0).tuples

[('The', 'DT', 'The'),
 ('Picture', 'NN', 'Picture'),
 ('of', 'IN', 'of'),
 ('Dorian', 'JJ', 'Dorian'),
 ('Gray', 'NNP', 'Gray'),
 ('By', 'IN', 'By'),
 ('Oscar', 'NNP', 'Oscar'),
 ('Wilde', 'NNP', 'Wilde')]

In [16]:
print(repr(text2))

Text(
	name='StringIO',
	encoding='None',
	nsents=1,
	nwords=8,
	nlemmas=8
)


## А теперь создадим корпус из коллекции документов

In [14]:
# MODULEDIR это путь до директории каталога nlptk относительно которого мы получаем путь к коллекции документов
source = os.path.abspath(os.path.join(nlptk.MODULEDIR,r'corpus\en'))
corpus = Corpus(Path(source,"*.txt"), prep, clean, filters)
#corpus.saveas = ("txt","pickle") # по умолчанию сохраняется в обоих форматах
# но загружаться будет согласно тому, что указано в loadas
corpus.loadas = "pickle"
#corpus.verbose = True  # при True будет выводить небольшую отладочную информацию об этапах обработки каждого документа
#corpus.rewrite = True # перетегировать все тексты, даже если они сохранены 
#corpus.filtrate = True # применить фильтры при формировании общего вокабуляра; по умолчанию False
print(help(corpus.tfidf))


Help on method tfidf in module nlptk.mining.text:

tfidf(n_doc, token=None, texts: List[List[str]] = [], sort=False, top=0) method of nlptk.mining.text.Corpus instance

None


In [15]:
texts = []

for text in corpus:
    # можно выводить объекты текстов text и что-то с ними делать;  по умолчанию объекты  Text генераторы предложений,
    # но класс Corpus передает им параметр inplace=True, что заставляет  их отработать на месте 
    texts.append(text) # в списке texts будут уже отработанные экзмепляры, а не генераторы
    #for i,sent in enumerate(text):
    #    sent # можно выводить предложения при обработке, но это замедлит работу 
        
print('Done!')


Done!


In [22]:
print(repr(corpus))

Corpus(
	names=['Austin Pride and Prejudice.txt', 'bronte_jane_txt.txt',...],
	ndocs=21,
	nwords=1923504,
	nlemmas=44985,
	nhapaxes=18123
)


In [21]:
sum(corpus.ccfs().values())

1923504

In [10]:
# Последнее предложение в корпусе, которое осталось в этой переменной после цикла обработки
# сами предложения в объекте корпуса никак не сохраняются в целях экономии памяти. 
# Сохраняются только всевозможные частотные словари
#sent

In [10]:
# работаем с последним текстом, который интерпретатор сохранил в этой переменной
text

Text(
	name='wilde_picture_of_dorian_gray_txt',
	encoding='unknown',
	nsents=6504,
	nwords=79460,
	nlemmas=5785
)

In [16]:
model = texts[0].embed.train(texts[0].sents2lemmas(filtrate=True,lower=True),size=1000)

In [17]:
model.wv.most_similar(positive=['mother'])


[('might', 0.9999929070472717),
 ('lydia', 0.9999927878379822),
 ('though', 0.9999927878379822),
 ('family', 0.9999927282333374),
 ('without', 0.9999927282333374),
 ('give', 0.9999925494194031),
 ('manner', 0.9999925494194031),
 ('friend', 0.9999923706054688),
 ('soon', 0.9999923706054688),
 ('father', 0.999992311000824)]

In [10]:
model.wv.most_similar(positive=['father'])

[('might', 0.9999924898147583),
 ('first', 0.9999924898147583),
 ('sister', 0.9999924302101135),
 ('lydia', 0.9999923706054688),
 ('darcy', 0.999992311000824),
 ('would', 0.9999921321868896),
 ('manner', 0.9999920725822449),
 ('family', 0.9999920129776001),
 ('find', 0.9999919533729553),
 ('much', 0.9999919533729553)]

In [29]:
texts[3].nwords

104790

6794

In [18]:
sents =  texts[3].sents2words(filtrate=True, pos={"NN","NNP","VB","ADJ"}, lower=True)
sents2 = texts[3].sents2lemmas(filtrate=True,pos={"NN","NNP","VB","ADJ"}, lower=True)

In [12]:

sents[0]

['adventures', 'sherlock', 'holmes', 'arthur', 'conan', 'doyle']

In [19]:
model3 = texts[3].embed.train(sents, size=1000, iter=15, window=2, sg=1, seed=10)
model4 = texts[3].embed.train(sents, size=1000, iter=15, window=2, sg=1, seed=10)

In [14]:
model3.wv.most_similar(positive=['doctor'], topn=10)


[('account', 0.9999289512634277),
 ('danger', 0.9999247789382935),
 ('advice', 0.9999229907989502),
 ('george', 0.9999229311943054),
 ('walk', 0.9999223947525024),
 ('come', 0.9999217391014099),
 ('show', 0.9999217391014099),
 ('box', 0.9999212026596069),
 ('evidence', 0.9999212026596069),
 ('ventilator', 0.9999203681945801)]

In [20]:
model4.wv.most_similar(positive=['holmes'], topn=10)

[('friend', 0.9997193813323975),
 ('surprise', 0.9997183084487915),
 ('oclock', 0.9997113943099976),
 ('chair', 0.9997076392173767),
 ('silence', 0.9997074007987976),
 ('morning', 0.9996960163116455),
 ('shoulder', 0.999691367149353),
 ('armchair', 0.9996887445449829),
 ('christmas', 0.9996850490570068),
 ('ceiling', 0.9996834397315979)]

In [16]:
# объект text при печати на консоль отображается как текст аналогичный сохраненному на диске тегированному тексту
pprint(str(text).split("\n")[:10])

['THE⁄The⁄DT PICTURE⁄Picture⁄NN OF⁄Of⁄IN DORIAN⁄Dorian⁄JJ GRAY⁄Gray⁄NNP '
 'BY⁄By⁄IN OSCAR⁄Oscar⁄NNP WILDE⁄Wilde⁄NNP THE⁄The⁄DT PREFACE⁄Preface⁄NNP '
 'The⁄The⁄DT artist⁄artist⁄NN is⁄be⁄VBZ the⁄the⁄DT creator⁄creator⁄NN of⁄of⁄IN '
 'beautiful⁄beautiful⁄JJ things⁄thing⁄NNS',
 'To⁄To⁄TO reveal⁄reveal⁄VB art⁄art⁄NN and⁄and⁄CC conceal⁄conceal⁄VB '
 'the⁄the⁄DT artist⁄artist⁄NN is⁄be⁄VBZ art⁄art⁄JJ aim⁄aim⁄NN',
 'The⁄The⁄DT critic⁄critic⁄NN is⁄be⁄VBZ he⁄he⁄PRP who⁄who⁄WP can⁄can⁄MD '
 'translate⁄translate⁄VB into⁄into⁄IN another⁄another⁄DT manner⁄manner⁄NN '
 'or⁄or⁄CC a⁄a⁄DT new⁄new⁄JJ material⁄material⁄NN his⁄his⁄PRP$ '
 'impression⁄impression⁄NN of⁄of⁄IN beautiful⁄beautiful⁄JJ things⁄thing⁄NNS',
 'The⁄The⁄DT highest⁄high⁄JJS as⁄a⁄IN the⁄the⁄DT lowest⁄low⁄JJS form⁄form⁄NN '
 'of⁄of⁄IN criticism⁄criticism⁄NN is⁄be⁄VBZ a⁄a⁄DT mode⁄mode⁄NN of⁄of⁄IN '
 'autobiography⁄autobiography⁄NN',
 'Those⁄Those⁄DT who⁄who⁄WP find⁄find⁄VBP ugly⁄ugly⁄JJ meanings⁄meaning⁄NNS '
 'in⁄in⁄IN beautiful⁄beautiful

In [17]:
# получаем общее числу слов в тексте, но после фильтрации
len(text.words(filtrate=True))

34895

In [18]:
len(text.words(filtrate=False))

79460

In [19]:
# число слов в тексте без фильтрации
text.nwords

79460

In [20]:
text.count(words=True, uniq=True)

7049

In [21]:
text.count(words=False, uniq=True)

5785

In [22]:
len(text.trie)

79460

In [23]:
# считаем сколько всего слов есть в корпусе по всем документам
sum(corpus.ccfs().values())

1923504

In [ ]:
# ПРОВЕРЯЕМ ЧТО ПОДСЧЕТЫ ВХОЖДЕНИЙ СЛОВ ОДИНАКОВЫ КАК ДЛЯ ОБЪЕКТОВ ТЕКСТОВ, ТАК И НА УРОВНЕ ВОКАБУЛЯРА КОРПУСА

In [24]:
[(txt.filename,txt.nwords) for txt in texts]

[('Austin Pride and Prejudice.txt', 121779),
 ('bronte_jane_txt.txt', 187333),
 ('bronte_wuthering_txt.txt', 118103),
 ('doyle_the_adventures.txt', 104790),
 ('dreiser_sister.txt', 155595),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 2333),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 2415),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 2119),
 ('fitzgerald_great_gatsby_txt.txt', 49346),
 ('Franz Kafka - Metamorphosis.txt', 22167),
 ('John Steinbeck - Of Mice and Men.txt', 30743),
 ('kipling_jungle_book_txt.txt', 51379),
 ('london_white_txt.txt', 72421),
 ('stevenson_treasure_island_txt.txt', 69182),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 318367),
 ('stoker_dracula_txt.txt', 160645),
 ('twain_tom_sawyer_txt.txt', 72251),
 ('walter_scott_ivanhoe_txt.txt', 193243),
 ('wells_invisible_man_txt.txt', 49434),
 ('wells_war_of_the_worlds_txt.txt', 60399),
 ('wilde_picture_of_dorian_gray_txt.txt', 79460)]

In [25]:
# считаем число слов для каждого текст по сумме частот из каждого словаря cfs
# это должно совпасть с тем числом, которое каждый Text хранит в свойстве nwords
[(name, sum(corpus.cfs(n).values())) for n,name in enumerate(corpus.filenames())]

[('Austin Pride and Prejudice.txt', 121779),
 ('bronte_jane_txt.txt', 187333),
 ('bronte_wuthering_txt.txt', 118103),
 ('doyle_the_adventures.txt', 104790),
 ('dreiser_sister.txt', 155595),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 2333),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 2415),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 2119),
 ('fitzgerald_great_gatsby_txt.txt', 49346),
 ('Franz Kafka - Metamorphosis.txt', 22167),
 ('John Steinbeck - Of Mice and Men.txt', 30743),
 ('kipling_jungle_book_txt.txt', 51379),
 ('london_white_txt.txt', 72421),
 ('stevenson_treasure_island_txt.txt', 69182),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 318367),
 ('stoker_dracula_txt.txt', 160645),
 ('twain_tom_sawyer_txt.txt', 72251),
 ('walter_scott_ivanhoe_txt.txt', 193243),
 ('wells_invisible_man_txt.txt', 49434),
 ('wells_war_of_the_worlds_txt.txt', 60399),
 ('wilde_picture_of_dorian_gray_txt.txt', 79460)]

In [26]:
# объект предложения в виде экземпляра класса TaggedSentence; n это порядковый номер предложения
text.sents(0)

TaggedSentence(
	'THE⁄The⁄DT PICTURE⁄Picture⁄NN OF⁄Of⁄IN DORIAN⁄Dorian⁄JJ GRAY⁄Gray⁄NNP BY⁄By⁄IN OSCAR⁄Oscar⁄NNP WILDE⁄Wilde⁄NNP THE⁄The⁄DT PREFACE⁄Preface⁄NNP The⁄The⁄DT artist⁄artist⁄NN is⁄be⁄VBZ the⁄the⁄DT creator⁄creator⁄NN of⁄of⁄IN beautiful⁄beautiful⁄JJ things⁄thing⁄NNS',
	 n=0
)

In [27]:
# легко догадаться, что это число слов в предложении
print(text.sents(0).nwords)
print(len(text.sents(0).tokens()))
print(len(text.sents(0).words()))
print(len(text.sents(0).words(uniq=True)))
print(len(text.sents(0).lemmas(uniq=True)))


18
18
18
14
14


In [11]:
# а так мы получаем все предложения из текста не длиннее одного слова
text.sents(max_words=1)[:3]

[TaggedSentence(
 	'Hallward⁄NN⁄Hallward',
 	 n=177
 ), TaggedSentence(
 	'Gray⁄NN⁄Gray',
 	 n=212
 ), TaggedSentence(
 	'Harry⁄NNP⁄Harry',
 	 n=236
 )]

In [31]:
# Давайте усложним условия: не менее трех, но не более пяти слов
text.sents(min_words=3,max_words=5)[:3]

[TaggedSentence(
 	'This⁄This⁄DT is⁄be⁄VBZ a⁄a⁄DT fault⁄fault⁄NN',
 	 n=5
 ), TaggedSentence(
 	'For⁄For⁄IN these⁄these⁄DT there⁄there⁄EX is⁄be⁄VBZ hope⁄hope⁄NN',
 	 n=7
 ), TaggedSentence(
 	'That⁄That⁄DT is⁄be⁄VBZ all⁄all⁄DT',
 	 n=11
 )]

In [28]:
# А если мы хотим получить предложение в более информативном виде? Класс Token к вашим услугам.
# Метод tokens трансформирует предложение в список экземпляров этого класса.
text.sents(0).tokens(lower=True)

(Token(word='the', idx=0, pos='DT', lemma='the', nsent=0),
 Token(word='picture', idx=1, pos='NN', lemma='picture', nsent=0),
 Token(word='of', idx=2, pos='IN', lemma='of', nsent=0),
 Token(word='dorian', idx=3, pos='JJ', lemma='dorian', nsent=0),
 Token(word='gray', idx=4, pos='NNP', lemma='gray', nsent=0),
 Token(word='by', idx=5, pos='IN', lemma='by', nsent=0),
 Token(word='oscar', idx=6, pos='NNP', lemma='oscar', nsent=0),
 Token(word='wilde', idx=7, pos='NNP', lemma='wilde', nsent=0),
 Token(word='the', idx=8, pos='DT', lemma='the', nsent=0),
 Token(word='preface', idx=9, pos='NNP', lemma='preface', nsent=0),
 Token(word='the', idx=10, pos='DT', lemma='the', nsent=0),
 Token(word='artist', idx=11, pos='NN', lemma='artist', nsent=0),
 Token(word='is', idx=12, pos='VBZ', lemma='be', nsent=0),
 Token(word='the', idx=13, pos='DT', lemma='the', nsent=0),
 Token(word='creator', idx=14, pos='NN', lemma='creator', nsent=0),
 Token(word='of', idx=15, pos='IN', lemma='of', nsent=0),
 Token(

In [33]:
# Попробуем включить фильтрацию (это те самые rules_filter, которые мы передавали в объект Corpus, 
# но которые по умолчанию отключены параметром corpus.filtrate=False, 
# чтобы в объекте корпуса сохранялась статистика по всем словам
text.sents(0).tokens(lower=True,filtrate=True)

(Token(word='picture', idx=1, pos='NN', lemma='picture', nsent=0),
 Token(word='dorian', idx=3, pos='JJ', lemma='dorian', nsent=0),
 Token(word='gray', idx=4, pos='NNP', lemma='gray', nsent=0),
 Token(word='oscar', idx=6, pos='NNP', lemma='oscar', nsent=0),
 Token(word='wilde', idx=7, pos='NNP', lemma='wilde', nsent=0),
 Token(word='preface', idx=9, pos='NNP', lemma='preface', nsent=0),
 Token(word='artist', idx=11, pos='NN', lemma='artist', nsent=0),
 Token(word='creator', idx=14, pos='NN', lemma='creator', nsent=0),
 Token(word='beautiful', idx=16, pos='JJ', lemma='beautiful', nsent=0),
 Token(word='things', idx=17, pos='NNS', lemma='thing', nsent=0))

In [34]:
# все слова предложения с указанным номером
text.sents(0).words()

('the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface',
 'the',
 'artist',
 'is',
 'the',
 'creator',
 'of',
 'beautiful',
 'things')

In [35]:
# все леммы предложения с указанным номером
text.sents(0).lemmas()

('the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface',
 'the',
 'artist',
 'be',
 'the',
 'creator',
 'of',
 'beautiful',
 'thing')

In [18]:
# разтегированное представаление предложения
text.sents(0).tuples

[('THE', 'DT', 'The'),
 ('PICTURE', 'NN', 'Picture'),
 ('OF', 'IN', 'Of'),
 ('DORIAN', 'JJ', 'Dorian'),
 ('GRAY', 'NNP', 'Gray'),
 ('BY', 'IN', 'By'),
 ('OSCAR', 'NNP', 'Oscar'),
 ('WILDE', 'NNP', 'Wilde'),
 ('THE', 'DT', 'The'),
 ('PREFACE', 'NNP', 'Preface'),
 ('The', 'DT', 'The'),
 ('artist', 'NN', 'artist'),
 ('is', 'VBZ', 'be'),
 ('the', 'DT', 'the'),
 ('creator', 'NN', 'creator'),
 ('of', 'IN', 'of'),
 ('beautiful', 'JJ', 'beautiful'),
 ('things', 'NNS', 'thing')]

In [37]:
# слова предложения отфильтрованные по частям речи - сущ. (в ед. и мн. числе)
text.sents(0).words(pos={"NN","NNS"})

('picture', 'artist', 'creator', 'things')

In [16]:
# леммы предложения отфильтованные по частям речи - сущ. (в ед. и мн. числе)
text.sents(0).lemmas(pos={"NN","NNS"})


('picture', 'artist', 'creator', 'thing')

In [15]:
# все леммы чей postag начинается на N, то есть любые существительные
text.sents(0).lemmas(pos="N")


('picture', 'gray', 'oscar', 'wilde', 'preface', 'artist', 'creator', 'thing')

In [40]:
text.sents(0).lemmas(pos="JJ")
# имя dorian неверно определяется как прилагательное, но здесь играет роль неоднозначность слова, так как такое прилагательное 
# тоже существует в английском

('dorian', 'beautiful')

In [42]:
# Предложение как есть, но уже без пунктуации, так как она была удалена на этапе очистки текста фильтрами TextCleaner
text.sents(0).raw

'THE PICTURE OF DORIAN GRAY BY OSCAR WILDE THE PREFACE The artist is the creator of beautiful things'

In [29]:
text.words(filtrate=False, pos="N")[:10]

['the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface']

In [43]:
# Можно получить частотные словари по любой части речи
verbs = text.postags("VERB")
verbs

FreqDist({'be': 3566, 'have': 1587, 'do': 656, 'say': 388, 'go': 295, 'know': 261, 'come': 232, 'look': 212, 'make': 205, 'think': 199, ...})

In [14]:
# отсортировать по убыванию частоты
text.postags("VERB", sort=-1)[:10]

[('be', 3566),
 ('have', 1587),
 ('do', 656),
 ('say', 388),
 ('go', 295),
 ('know', 261),
 ('come', 232),
 ('look', 212),
 ('make', 205),
 ('think', 199)]

In [13]:
# выбрать top наиболее частотных 
text.postags("VERB",top=10)

['be', 'have', 'do', 'say', 'go', 'know', 'come', 'look', 'make', 'think']

In [15]:
#verbs.hapaxes()

In [46]:
adjectives = text.postags("ADJ")
adjectives

FreqDist({'dorian': 292, 'good': 138, 'own': 130, 'great': 82, 'little': 82, 'young': 78, 'other': 72, 'more': 65, 'such': 65, 'old': 62, ...})

In [11]:
possessive = text.postags("POS",sort=-1)[:10]
possessive

[]

In [12]:
# генерация skipgram из текста (имплементация из nltk)
for n,s in enumerate(text.skipgrams(3,2, filtrate=True, words=True)):
    print(s)
    if n > 10:
        break

('picture', 'dorian', 'gray')
('picture', 'dorian', 'oscar')
('picture', 'dorian', 'wilde')
('picture', 'gray', 'oscar')
('picture', 'gray', 'wilde')
('picture', 'oscar', 'wilde')
('dorian', 'gray', 'oscar')
('dorian', 'gray', 'wilde')
('dorian', 'gray', 'preface')
('dorian', 'oscar', 'wilde')
('dorian', 'oscar', 'preface')
('dorian', 'wilde', 'preface')


In [49]:
# генерация ngram из текста (имплементация из nltk)
for n,s in enumerate(text.ngrams(3, lower=False, pad_left=True, left_pad_symbol='<s>', pad_right=True, right_pad_symbol='</s>')):
    print(s)
    if n > 10:
        break

('<s>', '<s>', 'The')
('<s>', 'The', 'Picture')
('The', 'Picture', 'Of')
('Picture', 'Of', 'Dorian')
('Of', 'Dorian', 'Gray')
('Dorian', 'Gray', 'By')
('Gray', 'By', 'Oscar')
('By', 'Oscar', 'Wilde')
('Oscar', 'Wilde', 'The')
('Wilde', 'The', 'Preface')
('The', 'Preface', 'The')
('Preface', 'The', 'artist')


In [12]:
#[(n,corpus.cfs(n).get('said',0)) for n in range(corpus.ndocs)]
[(n,corpus.cfs(n,'said')) for n in range(corpus.ndocs)]

[(0, 401),
 (1, 583),
 (2, 375),
 (3, 486),
 (4, 1474),
 (5, 24),
 (6, 0),
 (7, 2),
 (8, 236),
 (9, 51),
 (10, 390),
 (11, 430),
 (12, 70),
 (13, 341),
 (14, 1289),
 (15, 567),
 (16, 356),
 (17, 1451),
 (18, 531),
 (19, 166),
 (20, 262)]

In [51]:
[(n,corpus.tfidf(n,'said')) for n in range(corpus.ndocs)]


[(0, 0.0001606587000381203),
 (1, 0.00015184012272679605),
 (2, 0.00015491826256349981),
 (3, 0.00022628132251497257),
 (4, 0.00046220445377899574),
 (5, 0.000501913390512803),
 (6, 0),
 (7, 4.6050178545948135e-05),
 (8, 0.00023334168410784996),
 (9, 0.00011225237391803286),
 (10, 0.0006189429797377777),
 (11, 0.0004083335719429296),
 (12, 4.7159131907323064e-05),
 (13, 0.00024048807466936964),
 (14, 0.0001975409562372919),
 (15, 0.0001722059390834945),
 (16, 0.00024040218743433045),
 (17, 0.00036634976795974964),
 (18, 0.000524084176355715),
 (19, 0.000134094393154286),
 (20, 0.0001608736850288346)]

In [13]:
# набор наиболее весомых по метрике TFIDF слов в каждом тексте
res = [(n,
        corpus.tfidf(n,sort=-1)[0:7],
        name
       ) 
     for n,name in enumerate(corpus.filenames())
]
# берем текст по индексу 3 - Приключения Шерлока Холмса
res[3]

(3,
 [('holmes', 0.013422744214411884),
  ('sherlock', 0.0021765760086349586),
  ('watson', 0.0015041389643427844),
  ('lestrade', 0.0011040352384148304),
  ('rucastle', 0.0011040352384148304),
  ('mccarthy', 0.0009587674438865632),
  ('simon', 0.0008975571169628696)],
 'doyle_the_adventures.txt')

In [29]:
# отображение слов из 3-го текста в порядковый номер в словаре tdidf
ordered_by_tfidf = {word:n for n,(word,_) in enumerate(corpus.tfidf(3,sort=-1))}

In [36]:
#ordered_by_tfidf

In [53]:
# как видим, хотя метрика и дает наибольшие веса словам важным для контекста произведения - именам главных персонажей,
# но очень быстро скатывается до уровня частоупотребляемых вместе с этими словами сокращений типа mr.
# чья документтная частота очень и очень высока 
# (но их можно удалять фильтрацией, которую мы на уровне корпуса не включали, поэтому все стоп-слова и остались на месте)

# документная частота слова
corpus.dfs('mr')

16

In [40]:
# 
ordered_by_tfidf['mr'] # на  8 месте если упоядочить слова по TFIDF 

8

In [32]:
ordered_by_tfidf['said']

83

In [54]:
# число вхождений в текст по индексу 3
corpus.cfs(3,'s')

12

In [55]:
# документная частота слова "holmes"
corpus.dfs('holmes')

1

In [56]:
# число вхождений в текст по индексу 3
print(corpus.cfs(3,'holmes'))   # число вхождений словоформы
print(texts[3].vocab['holmes']) # число вхождений лемм
print(texts[3].words(filtrate=False).count('holmes'))
print(texts[3].lemmas(filtrate=False).count('holmes'))

462
462
462
462


In [11]:
print(corpus.cfs(3,'investigation'))       # число вхождений словоформы
print(corpus.cfs(3,'investigations'))
print(texts[3].vocab.get('investigation')) # число вхождений леммы == числу вхождений словоформ, которые образуют данную лемму
print(texts[3].words(filtrate=False).count('investigation'))  # число вхождений словоформы
print(texts[3].lemmas(filtrate=False).count('investigation')) # число вхождений леммы

12
3
15
12
15


In [57]:
# уникальных лемм
texts[3].count(words=False,uniq=True)

6639

In [58]:
# уникальных слов (без учета регистра)
texts[3].count(words=True, uniq=True)

8132

In [59]:
# уникальных слов (c учетом регистра)
texts[3].count(words=True, uniq=True, lower=False)

8724

In [60]:
# всего слов с повторами вхождений
texts[3].count(words=True)

104790

In [61]:
texts[3].nwords

104790

In [9]:
# число ключей в дереве == числу всех слов с повторами
len(texts[3].trie)

104790

In [62]:
len(set(texts[3].words(lower=True)))

8132

In [63]:
len(set(texts[3].words(lower=False)))

8724

In [32]:
corpus.tfidf(6,"red")

0.00020721212951756239

In [33]:
corpus.cfs(3,'said')

486

In [34]:
corpus.dfs('said')

20

In [32]:
corpus.tfidf(3,'said')

0.00022664684706076194

In [23]:
corpus.tfidf(6,sort=-1)[:10]

[('prospero', 0.007551523202290425),
 ('courtiers', 0.0038881773578561016),
 ('waltzers', 0.0037757616011452125),
 ('mummer', 0.0037757616011452125),
 ('prince', 0.0036805490437174567),
 ('ebony', 0.0029662764061375003),
 ('musicians', 0.002413282532933419),
 ('revellers', 0.002413282532933419),
 ('suite', 0.002071538600240377),
 ('sable', 0.0020565044356389405)]

In [22]:
# тоже самое (только в виде упорядоченного списка) - с использованием параметра top
corpus.tfidf(6,top=10)

['prospero',
 'courtiers',
 'waltzers',
 'mummer',
 'prince',
 'ebony',
 'musicians',
 'revellers',
 'suite',
 'sable']

In [46]:
help(corpus.tfidf)

Help on method tfidf in module nlptk.mining.text:

tfidf(n_doc, texts: List[List[str]] = None, token=None, sort=False) method of nlptk.mining.text.Corpus instance



In [21]:
print(corpus.cfs(1,'rochester'))
print(sum(corpus.cfs(1).values()))
print(corpus.dfs('rochester'))
print(corpus.tf(1,'rochester'))
print(corpus.idf('rochester'))
print(corpus.tfidf(1,token='rochester'))


317
186971
1
0.0016954500965390355
3.044522437723423
0.005161835860953437


In [32]:
input("--cfc--")
pprint(corpus.cfs(0,sort=-1)[:10])
input("--ccfc--")
pprint(corpus.ccfs(sort=-1)[:10])
input("--dfc--")
pprint(corpus.dfs(sort=-1)[:10])
input("--tfidf--")

#for n in range(corpus.ndocs):
#    pprint(corpus.tfidf(n,sort=-1)[:10]) 
            

--cfc--
[('the', 4320),
 ('to', 4127),
 ('of', 3596),
 ('and', 3529),
 ('her', 2216),
 ('i', 2046),
 ('a', 1945),
 ('in', 1861),
 ('was', 1843),
 ('she', 1703)]
--ccfc--
[('the', 104005),
 ('and', 64431),
 ('of', 53145),
 ('to', 51249),
 ('a', 42386),
 ('i', 34454),
 ('in', 29525),
 ('he', 28953),
 ('was', 26894),
 ('it', 24706)]
--dfc--
[('over', 21),
 ('once', 21),
 ('moment', 21),
 ('a', 21),
 ('minutes', 21),
 ('by', 21),
 ('four', 21),
 ('not', 21),
 ('from', 21),
 ('loud', 21)]
--tfidf--


''

In [42]:
# общее число слов-одиночек
all_hapaxes = corpus.hapaxes()
len(all_hapaxes)

18114

In [43]:
# н-да, вот что значит не включать фильтрацию на корпусе ...
sorted(all_hapaxes)[:10]

['a-bleeding',
 'a-blowing',
 'a-buttin',
 'a-callin',
 'a-chaffin',
 'a-changing',
 'a-crossin',
 'a-doin',
 'a-doing',
 'a-done']

In [65]:
# получаем слова-одиночки для каждого текста в корпусе
hapaxes = [(path, corpus.hapaxes(n)) for n,path in enumerate(corpus.filenames())]
hapaxes[0][1][:10]

['austen',
 'rightful',
 'morris',
 'grown-up',
 'newcomers',
 'over-scrupulous',
 'vexing',
 'sarcastic',
 'develop',
 'solace']

In [66]:
# сколько у нас гапаксов на каждый текст
[(h[0],len(h[1])) for h in hapaxes]

[('Austin Pride and Prejudice.txt', 2476),
 ('bronte_jane_txt.txt', 5809),
 ('bronte_wuthering_txt.txt', 4281),
 ('doyle_the_adventures.txt', 3775),
 ('dreiser_sister.txt', 4567),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 545),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 585),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 384),
 ('fitzgerald_great_gatsby_txt.txt', 3200),
 ('Franz Kafka - Metamorphosis.txt', 1295),
 ('John Steinbeck - Of Mice and Men.txt', 1397),
 ('kipling_jungle_book_txt.txt', 2211),
 ('london_white_txt.txt', 3205),
 ('stevenson_treasure_island_txt.txt', 2971),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 8757),
 ('stoker_dracula_txt.txt', 4487),
 ('twain_tom_sawyer_txt.txt', 3707),
 ('walter_scott_ivanhoe_txt.txt', 5791),
 ('wells_invisible_man_txt.txt', 2927),
 ('wells_war_of_the_worlds_txt.txt', 3333),
 ('wilde_picture_of_dorian_gray_txt.txt', 3543)]

In [67]:
# гапаксы с вычислением по всем словоформам, результат должен быть идентичен коду выше 
[(txt.filename, len(txt.hapaxes(words=True))) for txt in texts]

[('Austin Pride and Prejudice.txt', 2476),
 ('bronte_jane_txt.txt', 5809),
 ('bronte_wuthering_txt.txt', 4281),
 ('doyle_the_adventures.txt', 3775),
 ('dreiser_sister.txt', 4567),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 545),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 585),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 384),
 ('fitzgerald_great_gatsby_txt.txt', 3200),
 ('Franz Kafka - Metamorphosis.txt', 1295),
 ('John Steinbeck - Of Mice and Men.txt', 1397),
 ('kipling_jungle_book_txt.txt', 2211),
 ('london_white_txt.txt', 3205),
 ('stevenson_treasure_island_txt.txt', 2971),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 8757),
 ('stoker_dracula_txt.txt', 4487),
 ('twain_tom_sawyer_txt.txt', 3707),
 ('walter_scott_ivanhoe_txt.txt', 5791),
 ('wells_invisible_man_txt.txt', 2927),
 ('wells_war_of_the_worlds_txt.txt', 3333),
 ('wilde_picture_of_dorian_gray_txt.txt', 3543)]

In [68]:
# а так гапаксы будут считаться по леммам и, поскольку словарь лемм для каждого текста уже имеется в экземпляре каждого текста, 
# вычисляется гораздо быстрее
[(txt.filename, len(txt.hapaxes())) for txt in texts]

[('Austin Pride and Prejudice.txt', 1843),
 ('bronte_jane_txt.txt', 4608),
 ('bronte_wuthering_txt.txt', 3175),
 ('doyle_the_adventures.txt', 2979),
 ('dreiser_sister.txt', 3634),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 486),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 498),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 339),
 ('fitzgerald_great_gatsby_txt.txt', 2575),
 ('Franz Kafka - Metamorphosis.txt', 998),
 ('John Steinbeck - Of Mice and Men.txt', 1097),
 ('kipling_jungle_book_txt.txt', 1705),
 ('london_white_txt.txt', 2525),
 ('stevenson_treasure_island_txt.txt', 2269),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 7255),
 ('stoker_dracula_txt.txt', 3537),
 ('twain_tom_sawyer_txt.txt', 2875),
 ('walter_scott_ivanhoe_txt.txt', 4502),
 ('wells_invisible_man_txt.txt', 2327),
 ('wells_war_of_the_worlds_txt.txt', 2680),
 ('wilde_picture_of_dorian_gray_txt.txt', 2808)]

In [69]:
ndoc = 3
print(len(texts[ndoc].words(filtrate=False)))

hp = texts[ndoc].hapaxes(words=True)
print(len(hp))
print(sorted(hp)[-110:len(hp)])
'ycuea' in hp

104790
3775
['weedy', 'weekly', 'weigh', 'weighing', 'weird', 'welcome', 'welcomed', 'well-cut', 'well-groomed', 'well-lit', 'well-nigh', 'well-nurtured', 'well-opened', 'well-remembered', 'wellington', 'westaway', 'westawayous', 'westbury', 'western', 'westphail', 'westward', 'wheal', 'wheel', 'wheeled', 'wherever', 'whims', 'whine', 'whined', 'whirling', 'whishing', 'whisper', 'whispering', 'white-aproned', 'white-counterpaned', 'whiten', 'whiter', 'whither', 'whittington', 'wholesome', 'whoso', 'wicker-work', 'wicket', 'widow', 'wig', 'wight', 'wigmore', 'wigs', 'wilderness', 'wilful', 'wilhelm', 'will-o-the-wisp', 'willingly', 'willows', 'wilton', 'wimpole', 'win', 'winced', 'wincing', 'wind-swept', 'windfall', 'window-sill', 'windowsill', 'wine', 'wine-cellar', 'wines', 'winking', 'winter', 'wintry', 'wiry', 'wisdom', 'wishing', 'withdrawn', 'witnesses', 'wits', 'wives', 'woke', 'womanhood', 'won', 'wondered', 'wooded', 'wooden-leg', 'wooden-legged', 'worker', 'workmen', 'world-wi

True

In [70]:
print(sum(corpus.cfs(ndoc).values()))
ln = len(hapaxes[ndoc][1])
print(ln)
print(sorted(hapaxes[ndoc][1])[-110:ln])

104790
3775
['weedy', 'weekly', 'weigh', 'weighing', 'weird', 'welcome', 'welcomed', 'well-cut', 'well-groomed', 'well-lit', 'well-nigh', 'well-nurtured', 'well-opened', 'well-remembered', 'wellington', 'westaway', 'westawayous', 'westbury', 'western', 'westphail', 'westward', 'wheal', 'wheel', 'wheeled', 'wherever', 'whims', 'whine', 'whined', 'whirling', 'whishing', 'whisper', 'whispering', 'white-aproned', 'white-counterpaned', 'whiten', 'whiter', 'whither', 'whittington', 'wholesome', 'whoso', 'wicker-work', 'wicket', 'widow', 'wig', 'wight', 'wigmore', 'wigs', 'wilderness', 'wilful', 'wilhelm', 'will-o-the-wisp', 'willingly', 'willows', 'wilton', 'wimpole', 'win', 'winced', 'wincing', 'wind-swept', 'windfall', 'window-sill', 'windowsill', 'wine', 'wine-cellar', 'wines', 'winking', 'winter', 'wintry', 'wiry', 'wisdom', 'wishing', 'withdrawn', 'witnesses', 'wits', 'wives', 'woke', 'womanhood', 'won', 'wondered', 'wooded', 'wooden-leg', 'wooden-legged', 'worker', 'workmen', 'world-wi

In [71]:
texts[3].vocab['conan']

1

In [72]:
hapaxes[3][1][:10]

['conan',
 'doyle',
 'x',
 'eclipses',
 'predominates',
 'sex',
 'emotions',
 'abhorrent',
 'balanced',
 'softer']

In [29]:
corpus.cfs(3,'manifested')

1

In [74]:
# общекорпусная частота слова
corpus.ccfs('manifested')

28

In [73]:

print(corpus.cfs(0,'have'))
print(texts[0].words(filtrate=False).count('have'))

842
842


In [75]:
texts[0]

Text(
	name='Austin Pride and Prejudice',
	encoding='Windows-1252',
	nsents=5947,
	nwords=121779,
	nlemmas=4958
)

In [76]:
sum(corpus.cfs(0).values())


121779

In [77]:
corpus.ccfs("am")

3327

In [13]:
# частота слова для текста корпуса с номером 0
corpus.cfs(0,'vicious')

1

In [31]:
corpus.cfs(9,"gregor")

298

In [78]:
corpus.ccfs("s") # остатки от сокращений имен и названий у которых была удалена точка токенайзером

48

In [52]:
corpus.dfs('s')

8

In [24]:
# тегов притяжательных окончаний нет, так как nltk.tag_pos умеет их определять только в виде "'s", а всю пунктуацию 
# в начале и в конце токена стрипает токенайзер (точнее, обертка вокруг него), поэтому в тексте остаются только токены вида "s"
pos,_ = texts[10].postags("POS")
pprint(pos)


FreqDist({})


In [16]:
import nltk
print(nltk.pos_tag(["Gregor", "'s"]))
#[('Gregor', 'NNP'), ("'s", 'POS')]

print(nltk.pos_tag(["Gregor", "s"]))
#[('Gregor', 'NNP'), ('s', 'NN')]

# токены в которых притяжательное окончание присутствует tag_pos не умеет определять как NNP, то есть имена собственные
print(nltk.pos_tag(["Gregor's"]))
#[("Gregor's", 'NN')]


[('Gregor', 'NNP'), ("'s", 'POS')]
[('Gregor', 'NNP'), ('s', 'NN')]
[("Gregor's", 'NN')]


In [23]:
# выводим слова с сортировкой по возрастанию частот
corpus.cfs(9,sort=1)[:10]

[('troubled', 1),
 ('transformed', 1),
 ('vermin', 1),
 ('armour-like', 1),
 ('domed', 1),
 ('divided', 1),
 ('arches', 1),
 ('sections', 1),
 ('bedding', 1),
 ('cover', 1)]

In [27]:
texts[12].keywords(rating=('rake', dict(max_words=3))).topn(10)

['your stomach sour',
 'white fang hesitated',
 'goo night everybody',
 'hot words passed',
 'howd it happen',
 'give m time',
 'something was happening',
 'night had fallen',
 'one eye watched',
 'bring a light']

In [24]:
# много памяти для текстов имеющих больше 20 тыс. словоупотреблений... такой вот TextRank
print(texts[6].nwords) # рассказ Эдгара По 'Маска Красной Смерти'
texts[6].summarize(10, scores=False)

2415


['And now was acknowledged the presence of the Red Death',
 'Blood was its Avatar and its seal the redness and the horror of blood',
 'It was in this apartment also that there stood against the western wall a gigantic clock of ebony',
 'There were much of the beautiful much of the wanton much of the bizarre something of the terrible and not a little of that which might have excited disgust',
 'To and fro in the seven chambers there stalked in fact a multitude of dreams',
 'The figure was tall and gaunt and shrouded from head to foot in the habiliments of the grave',
 'And then for a moment all is still and all is silent save the voice of the clock',
 'But in spite of these things it was a gay and magnificent revel',
 'It was in the blue room where stood the prince with a group of pale courtiers by his side',
 'But these other apartments were densely crowded and in them beat feverishly the heart of life']

In [40]:
ndoc = 7
words =  []
for sent in texts[ndoc].sents():
    words.extend(sent.lemmas())
                     
len(words),  texts[ndoc].count(words=False)                   

(2119, 2119)

In [33]:
import itertools

def similarity(s1, s2):
        '''Мера сходства - коэффициент Сёренсена - 
        https://ru.wikipedia.org/wiki/Коэффициент_Сёренсена
        отношение количества одинаковых слов в 
        предложениях к суммарной длине предложений.
        ''' 
        if not len(s1) or not len(s2):
            return 0.0
        
        return len(s1.intersection(s2))/(1.0 * (len(s1) + len(s2)))


In [65]:
nsents = texts[ndoc].nsents    
pairs = itertools.combinations(range(nsents), 2)


In [66]:
scores = []
for i, j in pairs:
    sim = similarity(words[i], words[j])
    if sim:
        scores.append((i, j, sim)) 
    

print(len(scores))
print(scores[:10])

11378564
[(0, 1, 0.125), (0, 2, 0.15151515151515152), (0, 3, 0.125), (0, 4, 0.11538461538461539), (0, 5, 0.05555555555555555), (0, 6, 0.16666666666666666), (0, 7, 0.05263157894736842), (0, 8, 0.16), (0, 9, 0.08), (0, 10, 0.05)]


In [41]:
texts[0].trie('free').prefixes

['f', 'free']

In [80]:
texts[0].trie('free').startswith

['free',
 'free',
 'free',
 'free',
 'free',
 'free',
 'free',
 'freedom',
 'freedom',
 'freedom',
 'freedom',
 'freely',
 'freely',
 'freely',
 'freely']

In [81]:
texts[0].trie.startswith('free')

['free',
 'free',
 'free',
 'free',
 'free',
 'free',
 'free',
 'freedom',
 'freedom',
 'freedom',
 'freedom',
 'freely',
 'freely',
 'freely',
 'freely']

In [82]:
texts[0].trie('free').startswith

['free',
 'free',
 'free',
 'free',
 'free',
 'free',
 'free',
 'freedom',
 'freedom',
 'freedom',
 'freedom',
 'freely',
 'freely',
 'freely',
 'freely']

In [13]:
texts[0].trie.get('free', sort=True)

[(1101, 37),
 (1535, 4),
 (1670, 7),
 (2907, 19),
 (3761, 26),
 (4587, 20),
 (4935, 21)]

In [83]:
texts[3]

Text(
	name='doyle_the_adventures',
	encoding='ISO-8859-1',
	nsents=6794,
	nwords=104790,
	nlemmas=6639
)

In [12]:
set(texts[3].trie.startswith('miss'))

{'miss', 'missed', 'misses', 'missing', 'mission'}

In [54]:
texts[3].sents(6195)

TaggedSentence(
	'Man⁄NN⁄Man or⁄CC⁄or at⁄IN⁄at least⁄JJS⁄least criminal⁄JJ⁄criminal man⁄NN⁄man has⁄VBZ⁄have lost⁄VBN⁄lose all⁄DT⁄all enterprise⁄NN⁄enterprise and⁄CC⁄and originality⁄NN⁄originality',
	 n=6195
)

In [93]:
occurrences = texts[3].trie['crime']
occurrences

[(27, 11),
 (447, 19),
 (701, 37),
 (702, 21),
 (1051, 2),
 (1286, 13),
 (1566, 18),
 (1625, 5),
 (1625, 7),
 (1683, 6),
 (1832, 21),
 (2296, 14),
 (2585, 23),
 (2942, 12),
 (2952, 54),
 (3264, 13),
 (3265, 1),
 (3345, 39),
 (3347, 1),
 (3351, 26),
 (3447, 19),
 (3521, 9),
 (3796, 23),
 (4315, 14),
 (4388, 12),
 (5880, 24),
 (6180, 0),
 (6182, 10),
 (6188, 58),
 (6404, 24),
 (6407, 3),
 (6413, 57)]

In [87]:
# список предложений в которые входи слово "crime"
#[(nsent,texts[3].sents(nsent).raw) for nsent,_ in occurrences]

In [88]:
# узнаем в каких предложениях и на каких позициях употреблялось данное слово
texts[3].trie['holmes'][:10]

[(244, 0),
 (0, 4),
 (13, 2),
 (24, 5),
 (26, 31),
 (31, 36),
 (53, 2),
 (100, 7),
 (105, 1),
 (127, 0)]

In [89]:
texts[3].sents(24).words(5)

'holmes'

In [90]:
texts[3].sents(24).raw

'I had seen little of Holmes lately'

In [95]:
texts[0].sents(1103).raw

'The attention of the younger ones was then no longer to be gained by him'

In [35]:
#texts[1].trie['s']

In [106]:
len(texts[4].words(filtrate=False))

155595

In [105]:
texts[4].count()

155595

In [104]:
len(texts[4].trie)

155595

In [103]:
len(texts[0].words(uniq=True, lower=False))

6873

In [102]:
texts[0].count(uniq=True, lower=False)

6873

In [101]:
len(texts[0].words(uniq=True))

6387

In [41]:
texts[0].count(uniq=True)

6387

In [42]:
texts[0].count(words=False,uniq=True)

4958

In [107]:
len(texts[0].lemmas(uniq=True))

4958

In [108]:
[texts[i].nsents for i in range(corpus.ndocs)]

[5947,
 9709,
 6787,
 6794,
 12526,
 235,
 101,
 166,
 3420,
 780,
 3409,
 3207,
 4755,
 3730,
 18262,
 9694,
 4898,
 7341,
 3850,
 3287,
 6504]

In [98]:
verbs = texts[3].postags("VERB")
verbs

FreqDist({'be': 4507, 'have': 2105, 'say': 603, 'do': 586, 'come': 340, 'see': 322, 'know': 267, 'go': 248, 'think': 238, 'take': 214, ...})

In [97]:
texts[3].postags("VERB", sort=-1)[:10]

[('be', 4507),
 ('have', 2105),
 ('say', 603),
 ('do', 586),
 ('come', 340),
 ('see', 322),
 ('know', 267),
 ('go', 248),
 ('think', 238),
 ('take', 214)]

In [96]:
# получить ттегированное по частям речи представление прдложения
texts[3].sents(1)

TaggedSentence(
	'A⁄A⁄DT Scandal⁄Scandal⁄NNP in⁄in⁄IN Bohemia⁄Bohemia⁄NNP II⁄Ii⁄NNP',
	 n=1
)

In [100]:
texts[3].sents(1).tokens(lower=True)

(Token(word='a', idx=0, pos='DT', lemma='a', nsent=1),
 Token(word='scandal', idx=1, pos='NNP', lemma='scandal', nsent=1),
 Token(word='in', idx=2, pos='IN', lemma='in', nsent=1),
 Token(word='bohemia', idx=3, pos='NNP', lemma='bohemia', nsent=1),
 Token(word='ii', idx=4, pos='NNP', lemma='ii', nsent=1))

In [99]:
texts[3].keywords().topn(10)

['omne ignotum pro magnifico',
 'prima donna imperial opera',
 'botany variable geology profound',
 'clark russell fine sea-stories',
 'civilisation like untamed beasts',
 'british barque sophy anderson',
 'light mousseline de soie',
 'mrs st clair assertion',
 'san francisco cal u.s.a',
 'settee said holmes relapsing']

In [52]:

def test(ndoc):
    trie=sorted(texts[ndoc].trie().items(),key=lambda x:(x[1][0],x[1][1]))
    from collections import defaultdict
    sents = defaultdict(list)
    for w,(nsent,idx) in trie:
        tmp = idx
        sents[nsent].append(w)
        if tmp != idx:
            #TaggedSentence(' '.join(sents[nsent])
    return sents    
                            

In [19]:
#for ndoc in range(corpus.ndocs):
#    %time x=test(ndoc)

In [22]:
#result = []
#for ndocs in range(corpus.ndocs):
#    result.append((ndocs,texts[ndocs].trie('s',sort=-1))) 

In [18]:
#result

In [17]:

#for occur in result:
#    if occur[1] != []:
#        print(occur[0], texts[occur[0]].name)
#        for sent in occur[1]:
#            print(texts[occur[0]].sents(sent[0]).text, sent[0])